In [11]:
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *

# Set up environment
cc.setupEnvironment()
cc.listEnvironment()

spark = cc.startLocalCluster("weather_dim")
spark.getActiveSession()

cc.set_connectionProfile("veloDB")

df_operational_stations = (spark.read
    .format("jdbc")
    .option("driver", cc.get_Property("driver"))
    .option("url", cc.create_jdbc())
    .option("dbtable", "stations")
    .option("user", cc.get_Property("username"))
    .option("password", cc.get_Property("password"))
    .load())

zipcodes_iterator=(df_operational_stations
                        .select("zipcode").distinct()).toLocalIterator()
zipcodes = list(zipcodes_iterator)

print(zipcodes)


ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 2020
JPY_INTERRUPT_EVENT: 2020
JPY_PARENT_PID: 2028
JPY_SESSION_NAME: DWH_WeatherDim.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LOCALAP

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "weather_code", "rain"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["weather_code"] = hourly_weather_code
hourly_data["rain"] = hourly_rain

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)